# The Quinn family Secret Santa

Creates random pairing between participants under the constraints that no two members of the same group are matched and there is no repeat of a match from the past 5 years. Mails are sent to all participants automatically and anomynously.

## Setup

In [8]:
import pickle
from random import sample
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime

year = datetime.now().year
    

class Participant:

    def __init__(self, name, group, email):
        self.name = name
        self.group = group
        self.email = email
        
        
def get_recent_matches(participants, max_no_years=5):
    
    recent_matches = dict()
    for participant in participants:
        recent_matches[participant.name] = []

    for i in range(max_no_years):
            previous_year = year-1-i
            try:
                previous_pairing = pickle.load( open(f"previous/pairing{previous_year}.p", "rb" ) )
                for (giver, receiver) in previous_pairing.items():
                    recent_matches[giver].append(receiver)
            except:
                print(f"Missing file for {previous_year}")
                continue
                
    return recent_matches        
        

class Pairing:
    
    def __init__(self, participants):
        shuffled_participants = sample(participants, len(participants))
        self.pairs = dict(zip(participants, shuffled_participants))
        self.names = dict((giver.name, receiver.name) for (giver, receiver) in self.pairs.items())
        
    def is_valid(self):
        for (giver, receiver) in self.pairs.items():
            if giver.group == receiver.group: 
                return False
        return True
                
    def no_recent_matches(self, recent_matches):
        for (giver, receiver) in self.pairs.items():
            if receiver.name in recent_matches[giver.name]: 
                return False
        return True
        
                
class Mail:

    def __init__(self):
        self.port = 465
        self.smtp_server_domain_name = "smtp.gmail.com"
        self.sender_mail = "quinnsecretsanta@gmail.com"
        self.password = "dzkmxvrekiehcryi"
        self.ssl_context = ssl.create_default_context()
        self.service = smtplib.SMTP_SSL(self.smtp_server_domain_name, self.port, context=self.ssl_context)
        self.service.login(self.sender_mail, self.password)
    
    def quit(self):
        self.service.quit()
        
    def message(self, giver, receiver):      
        message = MIMEMultipart()
        message["Subject"] = str(year)+" Secret Santa!"
        message["From"] = self.sender_mail
        message["To"] = giver.email
        message.attach(MIMEText(message_text(giver.name, receiver.name)))
        return message.as_string()
    
    def trialmail(self, pairing):     
        for (giver, receiver) in pairing.pairs.items():
            if giver.name == "Eoin":
                self.service.sendmail(self.sender_mail, giver.email, self.message(giver, receiver))
                print(f"Mail sent to {giver.email}")
        
    def caution_sendmails_caution(self, pairing):     
        for (giver, receiver) in pairing.pairs.items():
            self.service.sendmail(self.sender_mail, giver.email, self.message(giver, receiver))
            print(f"Mail sent to {giver.email}")
            
def message_text(giver_name, receiver_name): 
    return f"""\
Dear {giver_name},

Congratulations, this year you are Secret Santa for {receiver_name}.

Happy searching!

Eolfin"""

## Participants

In [35]:
participants = [
    Participant("John", "group_John", "turboquinn@icloud.com"),
    Participant("Mary", "group_John", "familyquinn66@gmail.com"),
    Participant("Cathal", "group_Cathal", "cathal.quinn@gmail.com"),
    Participant("Iwona", "group_Cathal", "iwona.quinn@gmail.com"),
    Participant("Raghnall", "group_Raghnall", "rquinn@rossescommunityschool.ie"),
    Participant("Anne", "group_Raghnall", "ismiseanne@gmail.com"),
    Participant("Eamonn", "group_Eamonn", "eamonnquinn@yahoo.com"),
    Participant("Ellis", "group_Eamonn", "elliskelleher@gmail.com"),
    Participant("Eoin", "group_Eoin", "epquinn@gmail.com"),
    Participant("Maolíosa", "group_Maolíosa", "maoliosaq@yahoo.com"),
    Participant("Lisa", "group_Maolíosa", "lisadhanlon@gmail.com")
]

## Find pairing

In [59]:
recent_matches = get_recent_matches(participants, max_no_years=5)

counter = 1
counter_cutoff = 1000000

match = False
while match == False:        
    
    pairing = Pairing(participants)
    
    if pairing.is_valid() and pairing.no_recent_matches(recent_matches):
        print(f"Match found at attempt {counter}")
        match = True
        
    counter += 1
    if counter % 10000 == 0: print(counter)
    if counter == counter_cutoff: print("No match found"); break

10000
20000
Match found at attempt 28721


## Trial mail

In [22]:
mail = Mail()
mail.trialmail(pairing)
mail.quit()

Mail sent to epquinn@gmail.com


## Send out mails and save to file

Convert the following from markdown to code and run to send out the mails, and record the matches for later years.

pickle.dump(pairing.names, open(f"previous/pairing{year}.p", "wb"))
pickle.dump(pairing, open(f"previous/pairing_data_{year}.p", "wb"))

mail = Mail()

mail.caution_sendmails_caution(pairing)

mail.quit()